<img src="./images/wide_collage.png" width="80%"></img>
<img src="./images/drawing_collage.gif" width="80%"></img>

We are very pleased to announce the release of HoloViews 1.10!

This release contains a large number of features and improvements. Some highlights include:

**JupyterLab support**:

- Full compatibility with JupyterLab when installing the jupyterlab_holoviews extension ([#687](https://github.com/ioam/holoviews/issues/687))

**New components**:

- Added [``Sankey`` element](http://holoviews.org/reference/elements/bokeh/Sankey.html) to plot directed flow graphs ([#1123](https://github.com/ioam/holoviews/issues/1123))

- Added [``TriMesh`` element](http://holoviews.org/reference/elements/bokeh/TriMesh.html) and datashading operation to plot small and large irregular meshes ([#2143](https://github.com/ioam/holoviews/issues/2143))

- Added [``Chord`` element](http://holoviews.org/reference/elements/bokeh/Chord.html) to draw flow graphs between different nodes ([#2137](https://github.com/ioam/holoviews/issues/2137), [#2143](https://github.com/ioam/holoviews/pull/2143))

- Added [``HexTiles`` element](http://holoviews.org/reference/elements/bokeh/HexTiles.html) to plot data binned into a hexagonal grid ([#1141](https://github.com/ioam/holoviews/issues/1141))

- Added [``Labels`` element](http://holoviews.org/reference/elements/bokeh/Labels.html) to plot a large number of text labels at once (as data rather than as annotations) ([#1837](https://github.com/ioam/holoviews/issues/1837))

- Added [``Div`` element](http://holoviews.org/reference/elements/bokeh/Div.html) to add arbitrary HTML elements to a Bokeh layout ([#2221](https://github.com/ioam/holoviews/issues/2221))

- Added [``Violin`` element](https://holoviews.org/reference/elements/bokeh/Violin.html#bokeh-gallery-violin) to plot and compare distributions as kernel density estimates ([#2114](https://github.com/ioam/holoviews/pull/2114))

- Added [``PointDraw``](http://holoviews.org/reference/streams/bokeh/PointDraw.html), [``PolyDraw``](http://holoviews.org/reference/streams/bokeh/PolyDraw.html), [``BoxEdit``](http://holoviews.org/reference/streams/bokeh/BoxEdit.html), and [``PolyEdit``](http://holoviews.org/reference/streams/bokeh/PolyEdit.html) streams to allow drawing, editing, and annotating glyphs on a Bokeh plot, and syncing the resulting data to Python ([#2268](https://github.com/ioam/holoviews/issues/2459))

Plus many other bug fixes, enhancements and documentation improvements. For full details, see the [Release Notes](https://github.com/ioam/holoviews/releases/tag/v1.10.0).

<hr>

If you are using [Anaconda](https://www.anaconda.com/downloads), HoloViews can most easily be installed by executing the command ``conda install -c ioam holoviews`` . Otherwise, use ``pip install holoviews``.

<hr>

In [97]:
#ignore
import os
import numpy as np
import pandas as pd
import holoviews as hv
import datashader as ds

hv.extension('bokeh', 'matplotlib', logo=False, inline=False)

path = os.path.join(os.path.dirname(hv.__file__), '../examples/assets')
ds_path = os.path.join(os.path.dirname(ds.__file__), '../examples/data')

## JupyterLab support

With JupyterLab coming out of the alpha release stage we have finally made HoloViews compatible with JupyterLab by creating the [``jupyterlab_holoviews``](https://github.com/pyviz/jupyterlab_holoviews) extension, which can be installed with:

```
jupyter labextension install @pyviz/jupyterlab_holoviews
```

<img src="./images/jlab_screenshot.png" width="70%" style="display: table; margin: 0 auto;"></img>
    
It supports all the interactivity of the classic notebook. Separately deleting or re-executing a cell in the classic notebook or JupyterLab now cleans up the plot and ensures that any streams are unsubscribed, making it much easier to work with streaming plots.

## New elements

The main improvement in this release is the addition of a large number of elements. A number of these elements build on the ``Graph`` element introduced in the 1.9 release, including the ``Sankey``, ``Chord`` and ``TriMesh`` elements. Additionally ``HexTiles`` allow binning many points on a hexagonal grid, ``Violins`` allow comparing distributions across multiple variables, ``Labels`` can be used to plot vectorized text labels and ``Div`` allows displaying arbitrary HTML.

### Sankey

The new [``Sankey`` element](http://holoviews.org/reference/elements/bokeh/Sankey.html) is a pure-Python port of [d3-sankey](https://github.com/d3/d3-sankey), like most other elements it can be rendered using matplotlib and bokeh. In bokeh all the usual interactivity will be supported such as providing hover information and interactively highlighting connected nodes and edges. Here we have rendered the energy flow to SVG with matplotlib:

In [48]:
#ignore
%output backend='matplotlib' fig='svg'
hv.Sankey(pd.read_csv(path+'/energy.csv')).options(label_position='left', fig_size=350);

<a href="http://holoviews.org/gallery/demos/bokeh/energy_sankey.html#bokeh-gallery-energy-sankey"><img src="images/sankey.svg" style="display: table; margin: 0 auto;"></img></a>

### Chord

The [``Chord`` element](http://holoviews.org/reference/elements/bokeh/Chord.html) had been requested a number of times since it had been supported in the now deprecated bokeh-charts package. Thanks to bokeh's graph support hovering and tapping on the nodes highlights connected nodes, making sense of even densely interconnected graphs:

In [66]:
#ignore
%output backend='bokeh'
%opts Chord [edge_color_index='SourceID' label_index='City' color_index='AirportID' width=600 height=600]
%opts Chord (cmap='Category20' edge_cmap='Category20')

from bokeh.sampledata.airport_routes import routes, airports

hv.element.graphs.layout_chords.max_chords = 100

# Count the routes between Airports
route_counts = routes.groupby(['SourceID', 'DestinationID']).Stops.count().reset_index()
nodes = hv.Dataset(airports, 'AirportID', 'City')
chord = hv.Chord((route_counts, nodes), ['SourceID', 'DestinationID'], ['Stops'])

# Select the 20 busiest airports
busiest = list(routes.groupby('SourceID').count().sort_values('Stops').iloc[-20:].index.values)
busiest_airports = chord.select(AirportID=busiest, selection_mode='nodes')

mime = hv.renderer('bokeh').components(busiest_airports)
with open('images/chord.js', 'w') as f:
    f.write(mime[0]['application/javascript'])

<script src="./images/chord.js"></script>

### TriMesh

Also building on the graph capabilities is the [``TriMesh`` element](http://holoviews.org/reference/elements/bokeh/TriMesh.html) allows defining arbitrary meshes from the simplices and nodes. This allows easily visualizing Delaunay triangulations and even very large meshes thanks to the [datashader](http://datashader.org/) support. Below we can see an example of a regular ``TriMesh`` colored by vertex value and an interpolated mesh of the Chesepeake Bay containing 1M triangles:

In [67]:
#ignore
from scipy.spatial import Delaunay
from holoviews.operation.datashader import datashade

# First create the x and y coordinates of the points.
n_angles = 24
n_radii = 8
min_radius = 0.25
radii = np.linspace(min_radius, 0.95, n_radii)

angles = np.linspace(0, 2*np.pi, n_angles, endpoint=False)
angles = np.repeat(angles[..., np.newaxis], n_radii, axis=1)
angles[:, 1::2] += np.pi/n_angles

x = (radii*np.cos(angles)).flatten()
y = (radii*np.sin(angles)).flatten()
z = (np.cos(radii)*np.cos(angles*3.0)).flatten()
nodes = np.column_stack([x, y, z])

# Apply Delaunay triangulation
delauney = Delaunay(np.column_stack([x, y]))

# Mask off unwanted triangles.
xmid = x[delauney.simplices].mean(axis=1)
ymid = y[delauney.simplices].mean(axis=1)
mask = np.where(xmid*xmid + ymid*ymid < min_radius*min_radius, 1, 0)
simplices = delauney.simplices[np.logical_not(mask)]
nodes = hv.Points(nodes, vdims='z')

fpath = ds_path+'/Chesapeake_and_Delaware_Bays.3dm'
df = pd.read_table(fpath, delim_whitespace=True, header=None, skiprows=1,
                   names=('row_type', 'cmp1', 'cmp2', 'cmp3', 'val'), index_col=1)

e3t = df[df['row_type'] == 'E3T'][['cmp1', 'cmp2', 'cmp3']].values.astype(int) - 1
nd  = df[df['row_type'] == 'ND' ][['cmp1', 'cmp2', 'cmp3']].values.astype(float)
nd[:, 2] *= -1 # Make depth increasing

verts = pd.DataFrame(nd,  columns=['x', 'y', 'z'])
tris  = pd.DataFrame(e3t, columns=['v0', 'v1', 'v2'])

tiles = gv.WMTS('https://maps.wikimedia.org/osm-intl/{Z}/{X}/{Y}@2x.png')

points = gv.operation.project_points(gv.Points(verts, vdims=['z']))

trimesh_example = hv.TriMesh((simplices, nodes)).options(
    filled=True, edge_color_index='z', width=350, height=400, tools=['hover'],
    inspection_policy='edges', cmap='viridis', xaxis=None, yaxis=None, show_frame=False
) +\
tiles * datashade(hv.TriMesh((tris, points)), aggregator=ds.mean('z'), precompute=True).options(width=400, height=400)

mime = hv.renderer('bokeh').components(trimesh_example)
with open('images/trimesh.js', 'w') as f:
    f.write(mime[0]['application/javascript'])

<script src="./images/trimesh.js"></script>

### HexTiles

Another often requested feature is the addition of a hexagonal bin plot, which can be very helpful in visualizing large collections of points. Thanks to the recent addition of a hex tiling glyph in the [bokeh 0.12.15 release](https://bokeh.github.io/blog/2018/3/29/release-0-12-15/) it was straightforward to add this support in the form a [``HexTiles`` element]((http://holoviews.org/reference/elements/bokeh/HexTiles.html), which supports both simple bin counts and weighted binning.

Below we can see a ``HexTiles`` plot of ~7 million points representing the NYC population, where each hexagonal bin is scaled and colored by the bin value:

In [70]:
#ignore
import geoviews as gv
import dask.dataframe as dd
import geopandas as gpd
from cartopy import crs as ccrs

extents = (-8243208.293241908, 4953519.586791799, -8215378.42054359, 4997602.857167657)
(x0, y0, x1, y1) = extents

census = dd.read_parquet('/Users/philippjfr/datashader/examples/data/census.parq/')
df = hv.Dataset(census).select(meterswest=(x0, x1), metersnorth=(y0, y1)).data.compute()

In [71]:
#ignore
hextile_example = gv.WMTS('https://server.arcgisonline.com/ArcGIS/rest/services/World_Imagery/MapServer/tile/{Z}/{Y}/{X}.jpg', extents=extents).options(alpha=0.5) *\
hv.HexTiles(df, ['meterswest', 'metersnorth'], extents=extents).options(
    size_index='Count', xaxis=None, yaxis=None, gridsize=(40, 60),
    width=400, height=600, tools=['hover'], line_color='black', line_width=1,
    title_format='NYC Population'
)

mime = hv.renderer('bokeh').components(hextile_example)
with open('images/hextile_example.js', 'w') as f:
    f.write(mime[0]['application/javascript'])

<script src="./images/hextile_example.js"></script>

### Violin

[``Violin`` elements](http://holoviews.org/reference/elements/bokeh/Violin.html) were one of the most frequently requested plot types since the (matplotlib-only) seaborn interface was deprecated from HoloViews. With this release a native implementation of violins was added, which allows comparing distributions across one or more independent variables:

In [44]:
#ignore
%output fig='png'
from  bokeh.sampledata.autompg import autompg

violin = hv.Violin(autompg, ['origin', ('yr', 'Year')], ('mpg', 'Miles per Gallon')).redim.range(mpg=(8, 45))

violin.select(yr=(70, 81.1)).options(width=900, height=400);

<a href="http://holoviews.org/gallery/demos/bokeh/autompg_violins.html"><img src="images/violin.png" style="display: table; margin: 0 auto;"></img></a>

### Radial HeatMap

Thanks to the contributions of [Franz Woellert](https://github.com/mansenfranzen) the existing ``HeatMap`` element has now gained support for radial heatmaps to plot quantities varying over some cyclic variable such as the day of the week or time of day. Below we can see how the number of Taxi rides changes over the course of a year:

In [46]:
#ignore
%output backend='bokeh' fig='png'
df_nyc = pd.read_csv(path+"/nyc_taxi.csv.gz", parse_dates=["Pickup_date"])

# create relevant time columns
df_nyc["Day & Hour"] = df_nyc["Pickup_date"].dt.strftime("%A %H:00")
df_nyc["Week of Year"] = df_nyc["Pickup_date"].dt.strftime("Week %W")
df_nyc["Date"] = df_nyc["Pickup_date"].dt.strftime("%Y-%m-%d")

heatmap = hv.HeatMap(df_nyc, ["Day & Hour", "Week of Year"], ["Pickup_Count", "Date"])

%opts HeatMap [radial=True width=600 height=600 yticks=None xmarks=7 ymarks=3 start_angle=np.pi*19/14]
%opts HeatMap [xticks=("Friday", "Saturday", "Sunday", "Monday", "Tuesday", "Wednesday", "Thursday") tools=['hover']]

heatmap;

<a href="http://holoviews.org/gallery/demos/bokeh/nyc_radial_heatmap.html#bokeh-gallery-nyc-radial-heatmap"><img src="images/radial_heatmap.png" style="display: table; margin: 0 auto;"></img></a>

### Labels

Another often requested feature was to add a number of labels to a plot in an efficient way rather than relying on the ``Text`` element. The ``Labels`` element provides useful options to annotate an existing plot but can also be used on its own to plot unicode characters as in this example of t-SNE dimensionality reductions as applied to emoji word embeddings, which clusters semantically similar emojis together:

In [60]:
#ignore
%output fig='png'
emoji_df = pd.read_csv(path+'/emoji_embeddings.csv', index_col=0)
emojis = hv.Labels(emoji_df, label='Emoji t-SNE Embeddings')
emojis.options(width=800, height=500, xaxis=None, show_frame=False, yaxis=None).redim.range(x=(-30, 20), y=(-20, 20));

<a href="http://holoviews.org/gallery/demos/bokeh/emoji_tsne.html"><img src="images/emoji_labels.png" style="display: table; margin: 0 auto;"></img></a>

### Div

The ``Div`` element is exclusive to bokeh and allows embedding arbitrary HTML in a bokeh plot, e.g. it may be used to display pandas summary tables alongside a plot:

In [95]:
#ignore
%output fig='html' 
%opts Bars [width=650, height=400, tools=['hover'], xrotation=90, show_legend=False] (cmap='Category10')

macro_df = pd.read_csv('http://assets.holoviews.org/macro.csv', '\\t', engine='python')
bars = hv.Bars(macro_df, ['year', 'country'], [('gdp', 'GDP Growth %'), ('unem', 'Unemployment %')]).select(
    country=['France', 'Italy', 'United Kingdom', 'West Germany'], year=(1970, 1980)
).sort(['country', 'year'])

df = bars.data.set_index('year')

In [96]:
bars + hv.Div(df.describe().to_html())

:Layout
   .Bars.I :Bars   [year,country]   (gdp,unem)
   .Div.I  :Div

## Editing Tools

In the bokeh 0.12.15 release a new set of interactive tools were added to edit and draw different glyph types. These are now available from HoloViews as  the [``PointDraw``](http://holoviews.org/reference/streams/bokeh/PointDraw.html), [``PolyDraw``](http://holoviews.org/reference/streams/bokeh/PolyDraw.html), [``BoxEdit``](http://holoviews.org/reference/streams/bokeh/BoxEdit.html), and [``PolyEdit``](http://holoviews.org/reference/streams/bokeh/PolyEdit.html) streams classes, which make the drawn or edited data available to work with from Python, opening up the possibility for very complex interactivity and annotations.

<div style="display: table;">
<a href="http://holoviews.org/reference/streams/bokeh/PolyEdit.html"><img src="./images/poly_edit.gif" style="float: left" width="250px"></img></a>
<a href="http://holoviews.org/reference/streams/bokeh/BoxEdit.html"><img src="./images/box_edit.gif" style="float: left" width="250px"></img></a>
<a href="http://holoviews.org/reference/streams/bokeh/PolyDraw.html"><img src="./images/poly_draw.gif" style="float: left" width="250px"></img></a>
<a href="http://holoviews.org/reference/streams/bokeh/PointDraw.html"><img src="./images/point_draw.gif" style="float: left" width="250px"></img></a>
</div>

One such workflow might be to draw regions of interest on an image  using the ``BoxEdit`` and computing the the mean value over time:

<center><a href="http://holoviews.org/gallery/demos/bokeh/box_draw_roi_editor.html#bokeh-gallery-box-draw-roi-editor"><img src="./images/box_roi_editor_opt.gif" width="800px"></img></a></center>

## Setting options

The new ``.options`` method present on all viewable objects makes it much simpler to set options without worrying about the difference between plot, style and norm options. A comparison between the two APIs demonstrates how much easier to use the approach is:

In [91]:
#ignore
%output backend='bokeh' fig='png'
ls  = np.linspace(0, 10, 400)
x,y = np.meshgrid(ls, ls)
img = hv.Image(np.sin(x)*np.cos(y)+0.1*np.random.rand(400,400), 
               bounds=(-20,-20,20,20)).options(colorbar=True, xaxis=None, yaxis=None)

In [93]:
# New options API
img.options(cmap='RdBu_r', colorbar=True, width=360, height=300)

# Old opts API
img.opts(plot=dict(colorbar=True, width=360), style=dict(cmap='RdBu_r'));

Additionally it is now possible to explicitly declare which backend the options apply to making it straightforward to apply options for different backends, e.g.:

In [ ]:
img.options(width=360, backend='bokeh').options(fig_inches=(6, 6), backend='matplotlib')

## Data interfaces

The data interfaces which underlie HoloViews ability to work natively with a variety of datastructures also saw further improvements.

#### Binned and irregular data

It is now possible to declare binned data and irregular data, which has allowed ``Histogram`` and ``QuadMesh`` to finally support data interfaces, marking the final stages of converting all element types to ``Dataset`` classes.

In [16]:
#ignore
%opts Histogram QuadMesh {+axiswise}

In [17]:
## Binned data
n = 20
x = np.arange(n+1)         # Linear bins
y = np.logspace(0, 2, n+1) # Log bins
z = x*x[np.newaxis].T

# Irregular data
coords = np.linspace(-1.5, 1.5, n)
X,Y = np.meshgrid(coords, coords)

Qx = np.cos(Y) - np.cos(X) # 2D coordinate array
Qz = np.sin(Y) + np.sin(X) # 2D coordinate array
Z = np.sqrt(X**2 + Y**2)

hv.Histogram((x, y)) + hv.QuadMesh((y, x, z)) + hv.QuadMesh((Qx, Qz, Z))

:Layout
   .Histogram.I :Histogram   [x]   (Frequency)
   .QuadMesh.I  :QuadMesh   [x,y]   (z)
   .QuadMesh.II :QuadMesh   [x,y]   (z)

#### Dask arrays

Additionally it is now possible to leverage dask arrays without using xarray, allowing operations on the data to be performed out-of-core simply by annotating the data with coordinates:

In [ ]:
import dask.array as da

n = 100
dask_array = da.from_array(np.random.rand(n, n), chunks=10)
hv.Image((range(n), range(n), dask_array))

## Documentation & other improvements

A new **[Colormap](http://holoviews.org/user_guide/Colormaps.html)** user guide provides an overview of the available colormaps and how to effectively choose a colormap to reveal your data. It also introduces the new ``hv.plotting.list_cmaps`` function, which makes it easy to query for different kinds of colormaps, e.g. here is the output of ``hv.plotting.list_cmaps(category='Diverging', bg='light', reverse=False)`` when applied to an image:

In [80]:
#ignore
%output backend='matplotlib' fig='png'
hv.Layout([img.options(cmap=c, colorbar=False, sublabel_format='').relabel(c) 
           for c in hv.plotting.list_cmaps(category='Diverging', bg='light', reverse=False)])\
    .options(vspace=0.1, hspace=0.1).cols(7);

<a href="http://holoviews.org/user_guide/Colormaps.html"><img src="images/cmaps.png" style="display: table; margin: 0 auto;"></img></a>

Additionally a new **[Styling plots](http://holoviews.org/user_guide/Styling_Plots.html)** user guide provides an in depth overview on how to control colors, cycles, palettes and cmaps, which are now consistently handled across backends and support new features such as ``color_levels`` and ``symmetric`` color ranges:

In [73]:
#ignore
%output backend='bokeh' fig='png'
%opts Image (cmap='PiYG') [width=350, colorbar=True]

In [79]:
img.options(color_levels=5, symmetric=True) + img.options(color_levels=11, symmetric=True);

<a href="http://holoviews.org/user_guide/Styling_Plots.html"><img src="images/styling_plots.png" style="display: table; margin: 0 auto;"></img></a>